In [53]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from scipy.stats import bernoulli

%matplotlib inline

<img src="tex/tree1.png">

First, we assume a stock moves up a factor of $u$ or down a factor $d$ with a probability of $p$ or $(1-p)$, respectively.  At expiration, we know the price of the option.  It is either 0 if out-of-the-money, or its intrinsic value if in-the-money.

The price at the preceding node is then give by, 
$$C = e^{-rt}(pC_u + (1-p)C_d)$$

If the tree has multiple layers as shown below, we can then repeat the above process to entirely fill out the tree.

In [54]:
#  Manual process for simple two-step tree

#  Our initial stock price
S0 = 100

#  Time to expiration in years
t = 1

#  Assumed up and down percentages
u = 1.05
d = 1/u

#  Probability of an up move
p = 0.6

#  Stock price at expiration
S_u = u * S0
S_d = d * S0

#  Option strike price and risk-free rate
K = 102.5
r = 0.01

#  Call price at up node
C_u = S_u - K

#  Call price at down node
C_d = 0

#Print out the stock prices at expiration
#print(S_u, S_d)

#  Calculate and print the call price at t = 0
C = np.exp(-r * t) * ( p*C_u + (1-p)*C_d)
print(C)

1.485074750623752


A multi-level tree might look like:
<img src="tex/tree2.png">

In [55]:
#  Define the number of layers
N = 5

#  Time to expiration, the we rescale it to time per layer step
t = 1
t = t / (N - 1)

#  Iniital stock price, strike price, and risk-free rate
S0 = 100
K = 102.5
r = 0.01

#  Assume a volatility and calculate the size of an up move, down move, and probability
sigma = 0.4
u = np.exp(sigma * np.sqrt(t))
d = 1/u
p = (np.exp(r * t) - d) / (u - d)

#  Create some empty matrices to hold our stock and call prices.
stock_prices = np.zeros( (N, N) )
call_prices = np.zeros( (N, N) )

#  Put our initial price in the matrix
stock_prices[0,0] = S0

#  Fill out the remaining values
for i in range(1, N ):
    M = i + 1
    stock_prices[i, 0] = d * stock_prices[i-1, 0]
    for j in range(1, M ):
        stock_prices[i, j] = u * stock_prices[i - 1, j - 1]
 
#  Calculate the call price at expiration.  if the call price is less than zero, it is out-of-the-money so we replace those values with zero.
expiration = stock_prices[-1,:] - K
expiration.shape = (expiration.size, )
expiration = np.where(expiration >= 0, expiration, 0)

#  Set the last row of the call matrix to our expiration values
call_prices[-1,:] =  expiration

#  Backpropagate to filll out our tree
for i in range(N - 2,-1,-1):
    for j in range(i + 1):
        call_prices[i,j] = np.exp(-r * t) * ((1-p) * call_prices[i+1,j] + p * call_prices[i+1,j+1])

#plt.spy(call_prices)
print(call_prices[0,0])

14.709673831491694
